<a href="https://colab.research.google.com/github/matbot/LinearProgrammingPython/blob/master/MaximizingProfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@hw6p1
!pip install pulp
from pulp import *

from prettytable import PrettyTable

In [0]:
# Instantiate the problem
model = LpProblem("Maximizing Profit of Tie Production", LpMaximize)

# Construct the decision variables
Silk = LpVariable('Silk', lowBound=0, cat='Integer')
Poly = LpVariable('Polyester', lowBound=0, cat='Integer')
Blend1 = LpVariable('Blend1', lowBound=0, cat='Integer')
Blend2 = LpVariable('Blend2', lowBound=0, cat='Integer')

# Objective Function: Sum of profit from each type.
model += ((Silk*6.7 - Silk*.75 - Silk*(0.125*20)) + 
         (Poly*3.55 - Poly*.75 - Poly*(0.08*6)) +
         (Blend1*4.31 - Blend1*.75 - Blend1*(0.05*6 + 0.05*9)) +
         (Blend2*4.81 - Blend2*.75 - Blend2*(0.03*6 + 0.07*9)))

# Constraints
# Material availability constraints
model += Silk*.125 <= 1000, "Available silk"
model += Poly*.08 + Blend1*.05 + Blend2*.03 <= 2000, "Available polyester"
model += Blend1*.05 + Blend2*.07 <= 1250, "Available cotton"

# Production constraints
model += Silk >= 6000, "Min Silk"
model += Silk <= 7000, "Max Silk"
model += Poly >= 10000, "Min Poly"
model += Poly <= 14000, "Max Poly"
model += Blend1 >= 13000, "Min Blend1"
model += Blend1 <= 16000, "Max Blend1"
model += Blend2 >= 6000, "Min Blend2"
model += Blend2 <=  8500, "Max Blend2"

# Solve it
model.solve()

# Print LP Status
print("Tie Production Profit Maximization Status: {}".format(LpStatus[model.status]))

# Print tie types and optimal number.
x = PrettyTable()
x.field_names = ["Tie", "Number to Produce"]
x.add_row(["Silk", int(Silk.varValue)])
x.add_row(["Polyester", int(Poly.varValue)])
x.add_row(["Blend1", int(Blend1.varValue)])
x.add_row(["Blend2", int(Blend2.varValue)])
print(x)

# Print objective max profit.
print("Maximum Profit = ${}".format(round(pulp.value(model.objective),2)))

# Print constraint dual costs and slack
y = PrettyTable()
y.field_names = ["Name", "Constraint", "Slack"]
for name, c in model.constraints.items():
  y.add_row([name, c, c.slack])
print(y)

Tie Production Profit Maximization Status: Optimal
+-----------+-------------------+
|    Tie    | Number to Produce |
+-----------+-------------------+
|    Silk   |        7000       |
| Polyester |       13625       |
|   Blend1  |       13100       |
|   Blend2  |        8500       |
+-----------+-------------------+
Maximum Profit = $120196.0
+---------------------+------------------------------------------------------+---------+
|         Name        |                      Constraint                      |  Slack  |
+---------------------+------------------------------------------------------+---------+
|    Available_silk   |                 0.125*Silk <= 1000.0                 |  125.0  |
| Available_polyester | 0.05*Blend1 + 0.03*Blend2 + 0.08*Polyester <= 2000.0 |   -0.0  |
|   Available_cotton  |         0.05*Blend1 + 0.07*Blend2 <= 1250.0          |   -0.0  |
|       Min_Silk      |                     Silk >= 6000                     | -1000.0 |
|       Max_Silk      |    